In [2]:
import pandas as pd
import os

In [3]:
DATA_PATH_MALWARE = "../csv/malware"
DATA_PATH_GOODWARE = "../csv/goodware"
DATASET_PATH = "../csv"
col_names = [' Source IP', ' Source Port', ' Destination IP', ' Destination Port', ' Protocol', ' Timestamp',
             ' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets', 'Total Length of Fwd Packets',
             ' Total Length of Bwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
             ' Fwd Packet Length Std', 'Bwd Packet Length Max', ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
             ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std',
             ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
             ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min',
             'Fwd PSH Flags', ' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
             ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s', ' Min Packet Length', ' Max Packet Length',
             ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', 'FIN Flag Count', ' SYN Flag Count',
             ' RST Flag Count', ' PSH Flag Count', ' ACK Flag Count', ' URG Flag Count', ' CWE Flag Count',
             ' ECE Flag Count', ' Down/Up Ratio', ' Average Packet Size', ' Avg Fwd Segment Size', ' Avg Bwd Segment Size',
             'Fwd Header Length_1', 'Fwd Avg Bytes/Bulk', ' Fwd Avg Packets/Bulk', ' Fwd Avg Bulk Rate',
             ' Bwd Avg Bytes/Bulk', ' Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate', 'Subflow Fwd Packets',
             'Subflow Fwd Bytes', ' Subflow Bwd Packets', ' Subflow Bwd Bytes', 'Init_Win_bytes_forward',
             ' Init_Win_bytes_backward', ' act_data_pkt_fwd', ' min_seg_size_forward', 'Active Mean', ' Active Std',
             ' Active Max', ' Active Min', 'Idle Mean', ' Idle Std', ' Idle Max', ' Idle Min', 'Label']

# COSTRUZIONE DATASET
* avendo il dataset scomposto in molteplici *csv* vado ad unirli in uno singolo, almeno per un primo momento, distinguendo i files contenti traffico benevolo e malevolo

In [4]:
def CSV_merging(dest_df, files_path):
    for csv in os.listdir(files_path):
        #cols = pd.read_csv(f"{files_path}/{csv}", nrows=1)
        if not csv == ".DS_Store":
            print(csv)
            df = pd.read_csv(f"{files_path}/{csv}", encoding='utf-8', sep=',', engine='python')

            dest_df = pd.concat([dest_df, df])

    print('\n')

    df.close()
    return dest_df

In [ ]:
CSV_merging(pd.DataFrame(), DATA_PATH_MALWARE).to_csv(f"{DATASET_PATH}/malware.csv", index=False)
CSV_merging(pd.DataFrame(), DATA_PATH_GOODWARE).to_csv(f"{DATASET_PATH}/goodware.csv", index=False)

* eseguo labelling dei dati (MALWARE/BENIGN)

In [37]:
for csv in os.listdir(DATASET_PATH):
    if csv.endswith(".csv"):
        df = pd.read_csv(f"{DATASET_PATH}/{csv}")

        if "malware" in csv:
            df['Label'] = "MALWARE"
        else:
            df['Label'] = "BENIGN"

        df.to_csv(f"{DATASET_PATH}/{csv}", index=False)

df.close()

* unisco i due dataset creati al primo punto, in un singolo dataset

# NORMALIZZAZIONE DATASET
* rinomino i campi secondo le seguenti caratteristiche:
    * parole separate da spazio anzichè da '_';
    * prima lettera di ogni parola sarà maiuscola.
* accstare porta e IP sorgente (analogamete per la destinarione);
* eliminare punto da stringa che indica l'IP;
* cambiare tipi delle features, dove necessario:
    * *da fare lista*

Ultimi due passi sono necessari perchè algoritmi di ML accettano solo valori numerici.

In [3]:
def change_name_and_order(csv_path):
    new_names = list()
    data = pd.read_csv(csv_path)

    for col in data.columns:
        new_name = col.split('_')
        new_names.append(" ".join(new_name).title()) #funzione title trasforma la prima lettera di ogni parola in maiuscolo (capitalizza)
        #print(new_name)
    data.columns = new_names

    #riordinamento delle features
    data = data.loc[:, ["Src Ip", "Src Port", "Dst Ip", "Dst Port", "Protocol", "Timestamp", "Flow Duration", "Tot Fwd Pkts",
           "Tot Bwd Pkts", "Totlen Fwd Pkts", "Totlen Bwd Pkts", "Fwd Pkt Len Max", "Fwd Pkt Len Min", "Fwd Pkt Len Mean",
           "Fwd Pkt Len Std", "Bwd Pkt Len Max", "Bwd Pkt Len Min", "Bwd Pkt Len Mean", "Bwd Pkt Len Std", "Flow Byts S",
           "Flow Pkts S", "Flow Iat Mean", "Flow Iat Std", "Flow Iat Max", "Flow Iat Min", "Fwd Iat Tot", "Fwd Iat Mean",
           "Fwd Iat Std", "Fwd Iat Max", "Fwd Iat Min", "Bwd Iat Tot", "Bwd Iat Mean", "Bwd Iat Std", "Bwd Iat Max",
           "Bwd Iat Min","Fwd Psh Flags", "Bwd Psh Flags", "Fwd Urg Flags", "Bwd Urg Flags",
           "Fwd Header Len", "Bwd Header Len", "Fwd Pkts S", "Bwd Pkts S", "Pkt Len Min", "Pkt Len Max", "Pkt Len Mean",
           "Pkt Len Std", "Pkt Len Var", "Fin Flag Cnt", "Syn Flag Cnt", "Rst Flag Cnt", "Psh Flag Cnt", "Ack Flag Cnt",
           "Urg Flag Cnt", "Cwe Flag Count", "Ece Flag Cnt", "Down Up Ratio", "Pkt Size Avg", "Fwd Seg Size Avg",
           "Bwd Seg Size Avg", "Fwd Header Len", "Fwd Byts B Avg", "Fwd Pkts B Avg", "Fwd Blk Rate Avg", "Bwd Byts B Avg",
           "Bwd Pkts B Avg", "Bwd Blk Rate Avg", "Subflow Fwd Pkts", "Subflow Fwd Byts", "Subflow Bwd Pkts",
           "Subflow Bwd Byts", "Init Fwd Win Byts", "Init Bwd Win Byts", "Fwd Act Data Pkts", "Fwd Seg Size Min",
           "Active Mean", "Active Std", "Active Max", "Active Min", "Idle Mean", "Idle Std", "Idle Max", "Idle Min",
           "Label"]]

    data.to_csv(csv_path, index=False, header=col_names)
    data.close()

def normalize_ip(csv_path):
    data = pd.read_csv(csv_path)

    data.iloc[:, 0] = data.iloc[:, 0].map(lambda ip : __string_to_integer_ip(ip))
    data.iloc[:, 2] = data.iloc[:, 2].map(lambda ip : __string_to_integer_ip(ip))

    data.to_csv(csv_path, index=False)
    data.close()

def __string_to_integer_ip(ip):
    splitted_ip = ip.split('.')

    return "".join(splitted_ip)

def change_features_type(csv_path, features_types):
    data = pd.read_csv(csv_path)

    for i in range(data.shape[1]):
        data.iloc[:, i].astype(features_types[i])

    data.to_csv(csv_path, index=False)
    data.close()

Cambio nome delle features all'interno del dataset e cambio l'ordine di esse.

In [5]:
change_name_and_order(f"{DATASET_PATH}/malware.csv")
change_name_and_order(f"{DATASET_PATH}/goodware.csv")

Converto il formato stringa dell'indirizzo IP (aaa.bbb.ccc.ddd) in un formato comprensibile all'algoritmo di machine learning, cioè un intero nel seguente formato **aaabbbcccddd**.

In [6]:
normalize_ip(f"{DATASET_PATH}/malware.csv")
normalize_ip(f"{DATASET_PATH}/goodware.csv")

Unione dataset goodware e malware in un unico file in formato csv

In [5]:
df_malware = pd.read_csv(f"{DATASET_PATH}/malware.csv", low_memory=False)
df_goodware = pd.read_csv(f"{DATASET_PATH}/goodware.csv", low_memory=False)

pd.concat([df_malware, df_goodware]).to_csv(f"{DATASET_PATH}/dataset.csv", index=False)